In [5]:
import numpy as np
import itertools as itt
import time

## Trabalho 1 - Cálculo Numérico

Jorge Augusto Salgado Salhani

#USP: 8927418

### Questão 1

In [50]:
"""
Retorna decomposição da matriz A em LU utilizando eliminação de Gauss
"""
def func1(A):
  n = A.shape[0]
  U = A.copy()
  L = np.eye(n)

  # j: indice de iteração sobre colunas
  for j in range(n-1):

    # i: indice de iteração sobre linhas, ao longo do triangulo superior (i > j)
    for i in range(j+1, n):

      L[i,j] = U[i,j] / U[j,j]
      U[i, j:n] = U[i, j:n] - L[i,j] * U[j, j:n]

  return (L, U)

In [ ]:
# Teste 1: utilização da função func1
A = np.array([
  [2, -3, -1],
  [3, 2, -5],
  [2, 4, -1]
])

LU = func1(A)
L = LU[0]
U = LU[1]

In [ ]:
print('Matriz L = ')
print(L)
print()

print('Matriz U = ')
print(U)
print()

# Como validação, podemos fazer o retorno a A 
# de modo que A = LU
print('Original A = ')
print(A)
print()

print('Obtido LU = ')
print(np.matmul(L,U))

Matriz L = 
[[1.         0.         0.        ]
 [1.5        1.         0.        ]
 [1.         1.16666667 1.        ]]

Matriz U = 
[[ 2 -3 -1]
 [ 0  6 -3]
 [ 0  0  3]]

Original A = 
[[ 2 -3 -1]
 [ 3  2 -5]
 [ 2  4 -1]]

Obtido LU = 
[[ 2.  -3.  -1. ]
 [ 3.   1.5 -4.5]
 [ 2.   4.  -1.5]]


In [ ]:
"""
Retorna decomposição da matriz A em LU utilizando eliminação de Gauss
"""
def func2(A, p):
  n = A.shape[0]
  U = A.copy()
  L = np.eye(n)

  # j: indice de iteração sobre as colunas 
  for j in range(n-1):
    
    v = min(n, j+p+1)

    # i: indice de iteração sobre as linhas no triangulo superior (i > j)
    #  mas considerando como final o minimo entre n (linha toda)
    #  e v (valor ...)
    for i in range(j+1, v):
      L[i,j] = U[i,j] / U[j,j]
      U[i, j:v] = U[i, j:v] - L[i,j] * U[j, j:v]

  return (L, U)

In [59]:
# Teste 1: utilização da função func2
A = np.array([
  [2, -3, -1],
  [3, 2, -5],
  [2, 4, -1]
])

p = 1

LU = func2(A, p)
L = LU[0]
U = LU[1]

In [60]:
print('Matriz L = ')
print(L)
print()

print('Matriz U = ')
print(U)
print()

# Como validação, podemos fazer o retorno a A 
# de modo que A = LU
print('Original A = ')
print(A)
print()

print('Obtido LU = ')
print(np.matmul(L,U))

Matriz L = 
[[1.         0.         0.        ]
 [1.5        1.         0.        ]
 [0.         0.66666667 1.        ]]

Matriz U = 
[[ 2 -3 -1]
 [ 0  6 -5]
 [ 2  0  2]]

Original A = 
[[ 2 -3 -1]
 [ 3  2 -5]
 [ 2  4 -1]]

Obtido LU = 
[[ 2.         -3.         -1.        ]
 [ 3.          1.5        -6.5       ]
 [ 2.          4.         -1.33333333]]


Notamos que os resultados são compatíveis

A diferença entre as funções ocorre na quantidade de iterações e cálculos que são realizados para cada valor do triangulo superior da matriz A

In [61]:
n = 2000
p = 2
A = np.zeros((n,n))

for i in range(n):
  for j in range(max(0, i-p), min(n, i+p+1)):
    A[i,j] = np.random.normal()

In [66]:
start_time = time.time()
(L, U) = func1(A)
end_time = time.time()
print('Teste func1')
print(end_time - start_time)

print()

start_time = time.time()
print('Teste func2')
(L_, U_) = func2(A, p)
end_time = time.time()
print(end_time - start_time)

Teste func1
17.540777921676636

Teste func2
0.04382967948913574


In [67]:
# teste convergência via norma
print('Aproximação da solução via func1')
print(np.linalg.norm(L @ U - A))
print()
print('Aproximação da solução via func2')
print(np.linalg.norm(L_ @ U_ - A))

Aproximação da solução via func1
1.4752133367106018e-12

Aproximação da solução via func2
1.4752133367106018e-12


Como os resultados são compatíveis quando comparadas as suas normas em relação à solução real (A = LU) Notamos que a maior diferença ocorre na otimização de tempo de processamento

Isso se deve à redução do número de iterações da matriz triangular superior ...

## Questão 2

Definições das classes para resolução via métodos diretos e iterativos que serão úteis neste e na questão 3

In [94]:
class IterativeMethods:
  def __init__(self, max_it = 5000, epsilon = 0.001, printCG = False) -> None:
    self.MAX_ITERATION = max_it
    self.epsilon = epsilon
    self.printCG = printCG

  def solve_gauss_jacobi(self, A, b, x0):
    n = len(A)
    Dinv = np.diagflat([1/A[i,i] for i in range(n)])
    I = np.eye(n)
    
    C = I - np.matmul(Dinv, A)
    g = np.matmul(Dinv, b)

    if (self.printCG):
      print('C = ')
      print(C)
      print()

      print('g = ')
      print(g)
    
    norm = np.linalg.norm(b - np.matmul(A,x0))

    k = 0
    while norm > self.epsilon and k <= self.MAX_ITERATION:
      x0 = np.matmul(C,x0) + g
      k += 1
      norm = np.linalg.norm(np.matmul(A,x0) - b)
    
    if k >= self.MAX_ITERATION:
      raise Exception('Cálculo não converge')
    
    return x0

  # TODO não convergindo
  def solve_gauss_seidel(self, A, b, x0):
    n = len(A)
    L = np.tril(A)
    U = np.triu(A,1)
    
    Linv = - np.linalg.inv(L)
    C = - np.matmul(Linv, U)
    g = np.matmul(Linv, b)

    if (self.printCG):
      print('C = ')
      print(C)
      print()

      print('g = ')
      print(g)

    norm = np.linalg.norm(np.matmul(A,x0) - b)

    k = 0
    while norm > self.epsilon and k <= self.MAX_ITERATION:
      x0 = np.matmul(C,x0) + g
      k += 1
      norm = np.linalg.norm(np.matmul(A,x0) - b)
    
    if k >= self.MAX_ITERATION:
      raise Exception('Cálculo não converge')
    
    return x0
  
  def solve_gradientes(self, A, b, x0):
    n = len(A)
    r = b - np.matmul(A, x0)
    norm = np.linalg.norm(r)

    k = 0
    while norm > self.epsilon and k <= self.MAX_ITERATION:
      rT = r.transpose()
      Ar = np.matmul(A, r)
      alpha = np.dot(rT, r) / np.dot(r, Ar)
      x0 += alpha * r
      r = b - np.matmul(A, x0)
      norm = np.linalg.norm(r)
      
    if k >= self.MAX_ITERATION:
      raise Exception('Cálculo não converge')
    
    return x0

In [95]:
A = np.array([
  [4,-2,1,0,0,0],
  [-2,7,-2,1,0,0],
  [1,-2,7,-2,1,0],
  [0,1,-2,7,-2,1],
  [0,0,1,-2,7,-2],
  [0,0,0,1,-2,4]
])

b = np.array([-1,-2,1,1,-2,-1])


Questão 2a

In [96]:
itm = IterativeMethods(printCG=True)

In [97]:
x0 = np.array([0,0,0,0,0,0])

# Utilizando Gauss Jacobi
x = itm.solve_gauss_jacobi(A, b, x0)


C = 
[[ 0.          0.5        -0.25        0.          0.          0.        ]
 [ 0.28571429  0.          0.28571429 -0.14285714  0.          0.        ]
 [-0.14285714  0.28571429  0.          0.28571429 -0.14285714  0.        ]
 [ 0.         -0.14285714  0.28571429  0.          0.28571429 -0.14285714]
 [ 0.          0.         -0.14285714  0.28571429  0.          0.28571429]
 [ 0.          0.          0.         -0.25        0.5         0.        ]]

g = 
[-0.25       -0.28571429  0.14285714  0.14285714 -0.28571429 -0.25      ]


In [98]:
print("Solução Ax = b")
print("Vetor x = ")
print(x.transpose())

Solução Ax = b
Vetor x = 
[-0.50423043 -0.39819489  0.22117498  0.22117498 -0.39819489 -0.50423043]


In [99]:
# Validação de resultado correto
#  via comparativo da norma Ax - b = 0
print('Aproximação da solução via func2')
print(np.linalg.norm(A @ x - b))

Aproximação da solução via func2
0.0009438802773593731


In [100]:
x0 = np.array([0,0,0,0,0,0])

# Utilizando Gauss Seidel
try:
  itm.solve_gauss_seidel(A, b, x0)
except Exception as exc:
  print()
  print(exc)


C = 
[[-0.         -0.5         0.25       -0.         -0.         -0.        ]
 [-0.         -0.14285714 -0.21428571  0.14285714 -0.         -0.        ]
 [-0.          0.03061224 -0.09693878 -0.24489796  0.14285714 -0.        ]
 [-0.          0.02915452  0.00291545 -0.09037901 -0.24489796  0.14285714]
 [-0.          0.00395668  0.01468138  0.00916285 -0.09037901 -0.24489796]
 [-0.         -0.00531029  0.00661183  0.02717618  0.01603499 -0.15816327]]

g = 
[ 0.25        0.35714286 -0.07653061 -0.21574344  0.23500625  0.42143898]

Cálculo não converge


Questão 2b

In [101]:
itm = IterativeMethods(max_it=10000, epsilon=1E-8)

In [102]:
x0 = np.array([0,0,0,0,0,0])
xGJ = itm.solve_gauss_jacobi(A, b, x0)

x0 = np.array([1,0,0,0,0,0])
xGS = itm.solve_gauss_seidel(A, b, x0)

Exception: Cálculo não converge

Questão 3

Definição da classe para calculo de zeros de função

In [103]:
class FuncZeros:
  def __init__(self, max_it = 5000, epsilon = 0.001) -> None:
    self.MAX_ITERATION = max_it
    self.epsilon = epsilon

  def bissection(self, func, a, b):
    x = (a+b)/2
    erro = np.inf

    while erro > self.epsilon:
      if func(a) * func(x) < 0: b = x 
      else: a = x

      x0 = x
      x = (a+b)/2
      erro = abs(x-x0)

    return x
  
  def newton(self, func, dfunc, x):
    k = 0
    while k < self.MAX_ITERATION:
      dx = func(x) / dfunc(x)
      x -= dx
      print(x)
      if abs(dx) < self.epsilon:
        return x
      
    return None

Questão 3a

O momento em que a trajetória do projétil encontra o morro, conhecendo-se as funções que descrevem o morro e o projétil pode ser encontrada igualando ambas funções de modo que

$$
p(x) = -x^{4}+7.7x^{3}-18x^{2}+13.6x
$$

$$
q(x) = -x^{2}+5x+0.75
$$

$$
p(x) = q(x)
$$

Isso acontece quando

$$
-x^{2}+5x+0.75 = -x^{4}+7.7x^{3}-18x^{2}+13.6x
$$

Ou, reformulando para termos uma igualdade em 0

$$
-x^4 + 7.7x^3 - 17x^2 + 8.6x - 0.75 = 0
$$

Comotal função possui 4 raízes e queremos apenas o momento em que o projétil encontra o morro, queremos a solução para o intervalo [3,3.2] (conforme na figura na questão)

Apenas como comparativo, notamos que o valor que objetivamos está em $x \approx 3.173$, o que já valida o intervalo utilizado

![image.png](questao3a.png)

In [111]:
# max_it = 5: máximo de iterações
fz = FuncZeros(max_it=5, epsilon=0.001)

p = lambda x : -x**4+7.7*(x**3)-18*(x**2)+13.6*x
q = lambda x : -x**2+5*x+0.75

f = lambda x : -x**4 + 7.7*(x**3) - 17*(x**2) + 8.6*x - 0.75
a = 3
b = 3.2
x0 = fz.bissection(f, a, b)

print('Sendo a igualdade f(x0) = 0')
print('x0 =', x0)
print()
print('Sendo p(x0) altura')
print('p(x0) = ', p(x0))


Sendo a igualdade f(x0) = 0
x0 = 3.17265625

Sendo p(x0) altura
p(x0) =  6.546101510113125


Que conferem com os resultados esperados via análise gráfica

Questão 3 b